In [ ]:
from importlib import reload
from environment import lmmsabr
from environment.lmmsabr import make_nss_yield_df, compute_6m_forward_dataframe
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=10, suppress=True)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
reload(lmmsabr)




hed_expiry = 2
liab_expiry = 1

def generate_datasets(hed_expiry, liab_expiry): 



    hed_expiry_str = f"{int(12*hed_expiry)}m" if type(hed_expiry) == float else f"{hed_expiry}y"
    liab_expiry_str = f"{int(12*liab_expiry)}m" if type(liab_expiry) == float else f"{liab_expiry}y"
    lmm = lmmsabr.LMMSABR(imm=True, sub_steps=8, tenor=5, resolution=126, tau=0.5,sim_time=0.25,  swap_hedge_expiry=hed_expiry,swap_client_expiry=liab_expiry, beta=0.5)

    # Rebonato configuration
    rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
    theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
    phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}
    
    # fixed 5pct curve
    df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
    df[:] = 0.05
    # set up starting conditions
    lmm.sample_starting_conditions(df,
                                rho_kwargs=rho_config,
                                theta_kwargs=theta_config,
                                phi_kwargs=phi_config)
    lmm.prime()
    
    suffix = f"_flat_5pct_curve_{hed_expiry_str}_{liab_expiry_str}"
    lmm.generate_episodes_imm(n_episodes=50_000, min_stress_prob=0.0, max_stress_prob=0.0, out_dir=f'data/calm{suffix}')
    #lmm.generate_episodes_imm(n_episodes=50_000, min_stress_prob=1.0, max_stress_prob=1.0, out_dir=f'data/stress{suffix}')
    #lmm.generate_episodes_imm(n_episodes=45_000, min_stress_prob=0.0, max_stress_prob=0.8, out_dir=f'data/curriculum{suffix}') # only 20000, eval will be 50/50
    #lmm.generate_episodes_imm(n_episodes=5000,   min_stress_prob=0.5, max_stress_prob=0.5, out_dir=f'data/5050{suffix}')

the correlation will stay constant across regimes

In [ ]:
generate_datasets(0.5,1)
#generate_datasets(2,1)

In [ ]:
def make_plots(lmm):
    # lmm.simulate(seed=0)
    # lmm.get_swap_matrix()
    # lmm.get_sabr_params_imm()
    # lmm.plot(lmm.h_mat)
    # lmm.plot(lmm.g_mat)
    # print("f_sim")
    # lmm.plot(lmm.f_sim[:,lmm.ids_fwd_canon])
    # print("s_mat")
    # lmm.plot(lmm.k_mat[:,lmm.ids_fwd_canon]*lmm.g_mat[:,lmm.ids_fwd_canon])
    # plt.show()
    # plt.plot(lmm.V)
    # plt.show()
    # plt.plot(lmm.alpha)
    # plt.show()
    # plt.plot(lmm.phi)
    # plt.show()
    # n_sim = 100
    # swap_paths_hed = np.zeros((n_sim,lmm.swap_sim_shape[0]))
    # swap_paths_liab  = np.zeros((n_sim,lmm.swap_sim_shape[0]))
    # for i in range(n_sim):
    #     lmm.simulate()
    #     lmm.get_swap_matrix()
    #     swap_paths_hed[i] = lmm.swap_sim[:,0]
    #     swap_paths_liab[i] = lmm.swap_sim[:,1]
    # plt.plot(swap_paths_hed.T)
    # plt.show()
    # plt.plot(swap_paths_liab.T)
    # plt.show()
    struct = lmm.sabr_structured_data()
    #lt.plot(struct.hedge.swap.rate)
    print(np.std(np.diag(struct.hedge.swaption.vega)))
    plt.plot(np.diag(struct.hedge.swaption.vega))
    plt.show()

### Calm regime

Start by plotting the interesting features in the calm regime

In [ ]:
reload(lmmsabr)
hed_expiry_str = f"{int(12*hed_expiry)}m" if type(hed_expiry) == float else f"{hed_expiry}y"
liab_expiry_str = f"{int(12*liab_expiry)}m" if type(liab_expiry) == float else f"{liab_expiry}y"
lmm = lmmsabr.LMMSABR(imm=True, sub_steps=16, tenor=5, resolution=126, tau=0.5,sim_time=0.25,  swap_hedge_expiry=0.5,swap_client_expiry=1, beta=0.5)

# Rebonato configuration
rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}
#phi_config = {'phi_long':0,'phi_short':0, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}
#theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 200*0.0248, 200*0.0248)}
# fixed 5pct curve
df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
df[:] = 0.05
# set up starting conditions
lmm.sample_starting_conditions(df,
                            rho_kwargs=rho_config,
                            theta_kwargs=theta_config,
                            phi_kwargs=phi_config,
                            )
lmm.prime()
0
pnl_list = []
pnl_list_liab = []
#for i in range(20000):
lmm.simulate()
lmm.get_swap_matrix()
#pnl_list.append(lmm.swap_sim[:,0])
#pnl_list_liab.append()
#print(np.sum(np.array(pnl_list)),np.sum(np.array(pnl_list_liab)))
0

In [ ]:
diff_list = []
for i in range(20000):
    lmm.simulate()
    diff_list.append(lmm.f_sim[-1,lmm.ids_fwd_canon] *lmm.P[-1,lmm.ids_fwd_canon] -lmm.f_sim[0,lmm.ids_fwd_canon] *lmm.P[0,lmm.ids_fwd_canon])
diff_list = np.array(diff_list)
np.mean(diff_list, axis=0)

In [ ]:
lmm.rho_mat[lmm.ids_fwd_canon[-1],lmm.ids_fwd_canon]

In [ ]:
# simulate a bunch of paths, take the mean of the send to last forward, and the path of the last forward, mean them and plot
# simulate a bunch of paths, take the mean of the send to last forward, and the path of the last forward, mean them and plot
n_sims = 20000
# Pre-allocate numpy arrays - much more efficient than lists
time_steps = lmm.f_sim.shape[0]
swap_sim_mat = np.zeros((n_sims, time_steps))
for i in range(n_sims):
    lmm.simulate(seed=i)
    swap_sim_mat[i] = lmm.swap_sim[:,0]
    
#mean_second_last = np.mean(second_last_paths, axis=1)
mean_last = np.mean(swap_sim_mat, axis=0)
print(np.mean(swap_sim_mat))
#print(np.mean(second_last_paths[:,-1]))
#plt.plot(mean_second_last, label='mean second last')
plt.plot(mean_last, label='mean last')
plt.legend()
plt.title('Mean evolution of forward rates')
plt.xlabel('Time step')
plt.ylabel('Forward rate')
plt.show()

In [ ]:
lmm.simulate()
lmm.sabr_structured_data()
struct = lmm.sabr_structured_data()

In [ ]:
lmm.sabr_structured_data().hedge.swaption.price[:,0]

In [ ]:
n_sims = 10000
lmm.switch_regime(False)
swap_terminal_value_arr = np.zeros(n_sims)
swaption_terminal_value_arr_init = np.zeros((n_sims, lmm.swap_sim_shape[0]))
swaption_terminal_value_arr_mid = np.zeros((n_sims, lmm.swap_sim_shape[0]))
swaption_terminal_value_arr_end = np.zeros((n_sims, lmm.swap_sim_shape[0]))
swap_terminal_value_arr_manual = np.zeros(n_sims)
swap_value_arr = np.zeros((n_sims, lmm.swap_sim_shape[0]))
last_forward_arr = np.zeros((n_sims, lmm.swap_sim_shape[0]))
swap_pnl_arr = np.zeros((n_sims, lmm.swap_sim_shape[0]))
swaption_pnl_arr = np.zeros((n_sims, lmm.swap_sim_shape[0]))

for i in range(n_sims):
    struct = lmm.sabr_structured_data()
    lmm.simulate(seed=i)
    swap_terminal_value_arr[i] = np.maximum(struct.hedge.swap.value[-1], 0)
    swaption_terminal_value_arr_init[i] = struct.hedge.swaption.price[:,0] 
    swaption_terminal_value_arr_mid[i] = struct.hedge.swaption.price[:,31] 
    swaption_terminal_value_arr_end[i] = struct.hedge.swaption.price[:,62]
    swap_pnl_arr[i] = struct.hedge.swap.pnl[:,0]
    swap_value_arr[i] = struct.hedge.swap.value[:,0]
    last_forward_arr[i] = lmm.f_sim[:,lmm.ids_fwd_canon[-1]]
    swaption_pnl_arr[i] = struct.hedge.swaption.pnl[:,0]


In [ ]:
from scipy.stats import binned_statistic
slice_time = 62
swap_val_slice = swap_value_arr[:,slice_time]
swap_value_arr_init = swap_val_slice
swap_value_arr_mid = swap_val_slice - swap_value_arr[:,31]
swap_value_arr_end = swap_val_slice - swap_value_arr[:,62]
swaption_value_empirical = np.maximum(swap_value_arr[:,-1], 0)
swaption_value_empirical_init = swaption_value_empirical
swaption_value_empirical_mid = np.maximum(swap_value_arr[:,-1] - swap_value_arr[:,31], 0) 
swaption_value_empirical_end = np.maximum(swap_value_arr[:,-1] - swap_value_arr[:,62], 0)
swaption_price_init = swaption_terminal_value_arr_init[:,slice_time]
swaption_price_mid = swaption_terminal_value_arr_mid[:,slice_time] 
swaption_price_end = swaption_terminal_value_arr_end[:,slice_time] 
# construct strike dependent expected swaption value
strikes = np.arange(-0.1,0.15,0.01)
strike_val = [np.mean(np.maximum(swap_value_arr[:,-1]+K, 0)) for K in strikes]
# get indices for quantiles in the strikes array for the swap_value_arr
def plot_binned_mean(x, y, n_bins=10, label=None, linetype='-'):
    bin_means, bin_edges, _ = binned_statistic(
        x, y,
        statistic='mean',
        bins=n_bins
    )
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(bin_centers, bin_means, linetype, label=label)
    plt.legend()
    plt.xlabel('Strike offset (100 bps)')
    plt.ylabel('price')

plt.title('t=0 starting swaptions')
plot_binned_mean(swap_value_arr_init, swaption_price_init, label='analytic', linetype='o-')
plot_binned_mean(swap_value_arr_init, swaption_value_empirical_init, label='empirical')

plt.legend()
plt.show()
plt.title('t=31 starting swaptions')
plot_binned_mean(swap_value_arr_mid, swaption_price_mid, label='analytic', linetype='o-')
plot_binned_mean(swap_value_arr_mid, swaption_value_empirical_mid, label='empirical')
plt.legend()
plt.show()

plt.title('t=63 starting swaptions')
plot_binned_mean(swap_value_arr_end, swaption_price_end, label='analytic', linetype='o-')
plot_binned_mean(swap_value_arr_end, swaption_value_empirical_end, label='empirical')
plt.legend()
plt.show()

# plt.legend()
# plt.show()

# plt.scatter(swap_value_arr_init.T, swaption_terminal_value_arr_init[:,63]+np.maximum(swap_value_arr[:,-1], 0), alpha=0.5)
# plt.show()
# plt.scatter(swap_value_arr_mid.T, swaption_terminal_value_arr_mid[:,125]+np.maximum(swap_value_arr[:,-1], 0), alpha=0.5)
# plt.scatter(strikes, strike_val)
# plt.show()
# plt.scatter(swap_value_arr_end.T, (swaption_terminal_value_arr_end[:,125]+np.maximum(swap_value_arr[:,-1], 0)), alpha=0.5)
# plt.scatter(strikes, strike_val)
# plt.show()
# #plt.scatter(swap_value_arr_end.T, swaption_terminal_value_arr_end[:,32])



In [ ]:
# mean and std
plt.plot(np.mean(swap_value_arr, axis=0), label='mean swap value')
plt.fill_between(np.arange(0, swap_value_arr.shape[1]), 
                 np.mean(swap_value_arr, axis=0) - 1.96*np.std(swap_value_arr, axis=0)/np.sqrt(n_sims), 
                 np.mean(swap_value_arr, axis=0) + 1.96*np.std(swap_value_arr, axis=0)/np.sqrt(n_sims), 
                 alpha=0.2)

last_fwd_norm = np.mean(swaption_terminal_value_arr_init- swaption_terminal_value_arr_init[:,[0]], axis=0)
plt.plot(last_fwd_norm,'-o', label='mean swaption value')

plt.legend()
plt.title(f'Mean and std of swap value over {n_sims} simulations')
plt.axhline(0, color='black', linestyle='--')
plt.show()

In [ ]:
plt.plot(np.cumsum(np.mean(swaption_pnl_arr, axis=0)) , label='mean swaption value') #
plt.fill_between(np.arange(0, swap_value_arr.shape[1]), 
                 np.abs(np.mean(swaption_pnl_arr, axis=0))-np.abs(np.mean(swaption_price_init,axis=0))*0.005 - 1.96*np.std(swaption_pnl_arr, axis=0)/np.sqrt(n_sims), 
                 np.abs(np.mean(swaption_pnl_arr, axis=0))-np.abs(np.mean(swaption_price_init,axis=0))*0.005 + 1.96*np.std(swaption_pnl_arr, axis=0)/np.sqrt(n_sims), 
                 alpha=0.2)
plt.plot(np.abs(np.mean(swap_pnl_arr, axis=0))-0.0001 , label='mean swap value')
plt.fill_between(np.abs(np.arange(0, swap_value_arr.shape[1])), 
                 np.abs(np.mean(swap_pnl_arr, axis=0))- 0.0001 - 1.96*np.std(swap_pnl_arr, axis=0)/np.sqrt(n_sims), 
                 np.abs(np.mean(swap_pnl_arr, axis=0))- 0.0001 + 1.96*np.std(swap_pnl_arr, axis=0)/np.sqrt(n_sims), 
                 alpha=0.2)
plt.legend()
plt.axhline(0, color='black', linestyle='--')
plt.show()

In [ ]:
struct.hedge.swaption.pnl

In [ ]:
def plot_swaption_terminal_values(swaption_terminal_value_arr, n_sims):
    """
    Plots the mean swaption terminal values with confidence intervals.

    Args:
        swaption_terminal_value_arr (np.ndarray): Array of swaption terminal values.
        n_sims (int): Number of simulations to consider for confidence intervals.
    """
    plt.plot(np.mean(swaption_terminal_value_arr.T, axis=1), 
             label=f'analytic vs MC expected swaption price ({n_sims} simulations)')
    
    # Add confidence interval
    plt.fill_between(np.arange(swaption_terminal_value_arr.shape[1]),
                     np.nanmean(swaption_terminal_value_arr.T, axis=1) - 
                     np.nanstd(swaption_terminal_value_arr.T[:n_sims], axis=1) / np.sqrt(n_sims), 
                     np.nanmean(swaption_terminal_value_arr.T, axis=1) + 
                     np.nanstd(swaption_terminal_value_arr.T[:n_sims], axis=1) / np.sqrt(n_sims), 
                     alpha=0.2, label=f'confidence interval, {n_sims} simulations')
    
    # Make line at 0
    plt.axhline(0, color='red', linestyle='--', label='Zero line')
    plt.legend()
    plt.show()

# Example usage
plot_swaption_terminal_values(swaption_terminal_value_arr_init, 5000)
plot_swaption_terminal_values(swaption_terminal_value_arr_mid, 10000)
plot_swaption_terminal_values(swaption_terminal_value_arr_end, 20000)


In [ ]:
126/2

In [ ]:
#plt.plot(struct.hedge.swap.value)
plt.plot(struct.hedge.swap.value)
plt.show()

In [ ]:
plt.plot(lmm.sabr_structured_data().hedge.swap.rate, label='swap rate')
plt.show()

In [ ]:
plt.clf()
plt.plot(np.mean(swap_sim_mat-swap_sim_mat[0,0], axis=0), label='mean last')
plt.legend()
plt.title('Mean evolution of swap rates')
plt.xlabel('Time step')
plt.ylabel('Forward rate')
plt.show()

In [ ]:
f_last_paths.shape

In [ ]:
np.array([1e-5])

In [ ]:


plt.clf()
plt.plot(lmm.P[:,lmm.ids_fwd_canon[-2]], label='f_sim')
plt.plot( (1+ lmm.tau*lmm.f_sim[:,lmm.ids_fwd_canon[-1]]), label='f_sim*(1+tau*f_sim)')
plt.show()

In [ ]:
np.mean(last_paths), np.std(last_paths,ddof=1)/ np.sqrt(last_paths.shape[0])

In [ ]:

lmm.ids_fwd_canon[-2]

In [ ]:
lmm.P[:,lmm.ids_fwd_canon]

In [ ]:

print(lmm.P[0,lmm.ids_fwd_canon[1]]-lmm.P[0,lmm.ids_fwd_canon[2]])
print(lmm.tau*lmm.f_sim[0,lmm.ids_fwd_canon[2]]*lmm.P[0,lmm.ids_fwd_canon[2]])

In [ ]:

hed_expiry_str = f"{int(12*hed_expiry)}m" if type(hed_expiry) == float else f"{hed_expiry}y"
liab_expiry_str = f"{int(12*liab_expiry)}m" if type(liab_expiry) == float else f"{liab_expiry}y"
lmm = lmmsabr.LMMSABR(imm=True,tenor=1, resolution=126, tau=0.5,sim_time=127/(2*126),  swap_hedge_expiry=0.5,swap_client_expiry=1, beta=0.5)

# Rebonato configuration
rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}

# fixed 5pct curve
df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
df[:] = 0.05
# set up starting conditions
lmm.sample_starting_conditions(df,
                            rho_kwargs=rho_config,
                            theta_kwargs=theta_config,
                            phi_kwargs=phi_config)
lmm.prime()
lmm.simulate()
lmm.get_swap_matrix()
lmm.sabr_structured_data().hedge.swaption.price

In [ ]:

idx_swapt = np.arange(lmm.swap_sim_shape[0])


n_sim = 20000
swaption_val_0Tn = lmm.sabr_structured_data().hedge.swaption.price[idx_swapt,idx_swapt]
mc_mat = np.zeros((n_sim, len(swaption_val_0Tn)))
for i in range(n_sim):
    lmm.simulate()
    lmm.get_swap_matrix()
    P0T = lmm.P[:,-1]
    swaption_val_0Tn = lmm.sabr_structured_data().hedge.swaption.price[idx_swapt,idx_swapt]
    PTiTn = lmm.P[-1, -1] # last row is expiry, last column is terminal forward
    swap_value = lmm.annuity[:,0] *(lmm.swap_sim[:,0] - lmm.swap_sim[0,0] )
    swap_val_0Tn =   swap_value[-1] - swap_value
    H = np.maximum(swap_val_0Tn,0)
    MC_price = P0T* H / PTiTn
    mc_mat[i] = swaption_val_0Tn - MC_price




# TEST MONTE CARLO FOR VALUATION EFTER KØB; TJEK MONEYNESS VS BIAS PLOT BURDE LIGNE EN PARABEL

In [ ]:

lmm.simulate()
lmm.get_swap_matrix()

swaption_val_0Tn = lmm.sabr_structured_data().hedge.swaption.price[idx_swapt,idx_swapt]
np.mean(mc_mat,axis=0)
#np.std(mc_mat, axis=0)

In [ ]:
lmm.f_sim[:,::-1]

In [ ]:


pnl_sum = 0
for i in range(10000):
    lmm.simulate()
    lmm.get_swap_matrix()
    pnl_sum += np.sum(lmm.sabr_structured_data().hedge.swap.pnl)

pnl_sum

In [ ]:
lmm.sabr_structured_data().hedge.swap.value
atm_strikes = np.tile(lmm.swap_sim[:,0], (lmm.swap_sim.shape[0],1))

lmm.swap_sim[:,0] -atm_strikes[0,0]

In [ ]:
lmm.annuity

In [ ]:
T = lmm.f_sim.shape[0]
zcb = 1/(1 + np.diag((lmm.df_init['gamma'].values*lmm.df_init['mod_accrual'].values)[None,:T]*lmm.f_sim[:, lmm.df_init['i_s'][:T]]))
zcb[1]-zcb[0] 

In [ ]:
#array([ 0.0000008371, -0.0000031232, -0.0000001155, -0.0000019382, -0.0000012647,  0.0000006607, -0.0000011909, -0.0000073156, -0.0000064356, -0.0000061354, -0.0000106451, -0.0000138439, -0.0000095244, -0.000008958 , -0.0000077647, -0.0000088386, -0.0000100318, -0.0000128625, -0.0000115166, -0.0000105204, -0.0000113715, -0.0000073144, -0.0000076155, -0.0000073642, -0.0000084555, -0.0000112434, -0.0000090003, -0.0000090072, -0.0000080868, -0.0000070172, -0.0000077757, -0.0000053111, -0.0000063506, -0.0000085971, -0.000005859 , -0.0000045801, -0.0000037651, -0.0000042182, -0.0000054378, -0.0000061726, -0.0000069523, -0.0000103755, -0.000010628 , -0.000010634 , -0.0000120615, -0.0000103761, -0.0000062183, -0.0000059467, -0.0000065045, -0.0000058745, -0.0000053664, -0.0000032186, -0.0000043676, -0.0000029518,  0.0000002194,  0.000000012 , -0.0000017069, -0.0000064399, -0.000005785 , -0.0000062085, -0.0000044166, -0.0000076904, -0.0000104184, -0.0000078965, -0.0000082125, -0.0000039015,  0.0000009787,  0.0000008826,  0.0000025711,  0.0000044116,  0.0000043872,  0.0000033116,  0.0000055764,  0.0000029038,  0.0000030046,  0.0000029662,  0.000004167 ,  0.0000043804,  0.0000042176,  0.0000012407,  0.000004452 ,  0.0000055709,  0.0000056183,  0.000005853 ,  0.0000036503,  0.0000049367,  0.0000055624,  0.000007511 ,  0.0000080484,  0.000007072 ,  0.0000086943,  0.0000063446,  0.0000057801,  0.0000051984,  0.000005211 ,  0.0000065127,  0.000006563 ,  0.0000066182,  0.0000032482,  0.0000030445,  0.000003162 ,  0.0000019106,  0.0000021553,  0.0000014203,  0.0000018133,  0.0000025412,  0.0000021753,  0.0000037639,  0.000004185 ,  0.0000033238,  0.0000026754,  0.0000008248,  0.0000007841, -0.00000011  ,  0.0000016883,  0.0000002894,  0.0000008407, -0.0000001309, -0.0000017672, -0.0000043909, -0.0000040285, -0.0000029643, -0.0000013108, -0.0000009285, -0.0000012788, -0.0000033312,  0.    

In [ ]:
price_analytic- P0T*np.mean(np.array(mc_list))

In [ ]:
terminal_forward = 0
for i in range(100):
    lmm.simulate()
    terminal_forward += np.sum(lmm.f_sim[:,-1])


In [ ]:
f_sim_new = lmm.f_sim.copy()
f_sim_new[:,0] = f_sim_new=0.0
lmm.f_sim

### Stressed regime

In [ ]:

# lmm.switch_regime(stressed=False)
# lmm.simulate()
# lmm.get_swap_matrix()
# make_plots(lmm)

### Curriculum calm->stress regime

### 50/50 test set for evaluation across training methods